In [1]:
using LinearAlgebra
using Flux
using CUDA

In [2]:
CUDA.functional()

true

In [3]:
input = 60
middle = 128
output = 7
depth = 10

20

In [27]:
#model_cpu = Chain(Dense(input, middle), [Chain(Parallel(+, Chain(BatchNorm(middle), Dense(middle, middle, relu)),Dense(middle, middle, relu)), identity) for i in 1:depth], Base.Flatten , Parallel(vcat,Chain(Dense(middle, middle, tanh),Dense(middle, output)), Dense(middle, 1)))
model_cpu = Chain(Dense(input, middle), Tuple(Chain(Parallel(+, Chain(BatchNorm(middle), Dense(middle, middle, relu)),Dense(middle, middle, relu)), identity) for i in 1:depth)..., Flux.flatten , Parallel(vcat,Chain(Dense(middle, middle, tanh),Dense(middle, output)), Dense(middle, 1)))

Chain(
  Dense(60 => 128),                     # 7_808 parameters
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(128),                 # 256 parameters, plus 256
        Dense(128 => 128, relu),        # 16_512 parameters
      ),
      Dense(128 => 128, relu),          # 16_512 parameters
    ),
    identity,
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(128),                 # 256 parameters, plus 256
        Dense(128 => 128, relu),        # 16_512 parameters
      ),
      Dense(128 => 128, relu),          # 16_512 parameters
    ),
    identity,
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(128),                 # 256 parameters, plus 256
        Dense(128 => 128, relu),        # 16_512 parameters
      ),
      Dense(128 => 128, relu),          # 16_512 parameters
    ),
    identity,
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(128),                 # 256 parameters, plus 256
        Dense(128 => 

In [28]:
model_gpu = gpu(model_cpu)

Chain(
  Dense(60 => 128),                     # 7_808 parameters
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(128),                 # 256 parameters, plus 256
        Dense(128 => 128, relu),        # 16_512 parameters
      ),
      Dense(128 => 128, relu),          # 16_512 parameters
    ),
    identity,
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(128),                 # 256 parameters, plus 256
        Dense(128 => 128, relu),        # 16_512 parameters
      ),
      Dense(128 => 128, relu),          # 16_512 parameters
    ),
    identity,
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(128),                 # 256 parameters, plus 256
        Dense(128 => 128, relu),        # 16_512 parameters
      ),
      Dense(128 => 128, relu),          # 16_512 parameters
    ),
    identity,
  ),
  Chain(
    Parallel(
      +,
      Chain(
        BatchNorm(128),                 # 256 parameters, plus 256
        Dense(128 => 

In [17]:
input_cpu = rand(Float32, input, 100)
input_gpu = gpu(input_cpu)

60×100 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.738577  0.352467     0.687787  …  0.898021   0.434713   0.645
 0.887385  0.804706     0.91216      0.832005   0.046598   0.0181216
 0.563988  0.000522852  0.773837     0.930119   0.893516   0.279462
 0.40598   0.436399     0.496121     0.950423   0.251856   0.520948
 0.625389  0.256931     0.651062     0.0818405  0.761508   0.0659829
 0.274392  0.464407     0.327692  …  0.946897   0.270452   0.00340396
 0.445722  0.446448     0.546048     0.0785484  0.708661   0.427573
 0.85382   0.397422     0.190878     0.64371    0.233262   0.766823
 0.27824   0.629328     0.762274     0.90607    0.74769    0.462052
 0.074921  0.999979     0.169286     0.419686   0.229687   0.786957
 ⋮                                ⋱                        
 0.375196  0.106781     0.73321      0.913829   0.783116   0.520454
 0.900458  0.382878     0.264056     0.815667   0.809782   0.590929
 0.943172  0.258745     0.887859     0.587372   0.629634   0.993767
 0.4

In [32]:
@time model_cpu(input_cpu)

  0.007179 seconds (493 allocations: 6.112 MiB)


8×100 Matrix{Float32}:
  1.60501    1.67968    1.59362   …   1.54868    1.55584    1.64938
 -1.22871   -1.25437   -1.31036      -1.25169   -1.29198   -1.28681
 -1.78119   -1.79311   -1.86135      -1.81473   -1.84307   -1.86589
 -0.554376  -0.498259  -0.433132     -0.511761  -0.484417  -0.557348
  1.29387    1.28261    1.42652       1.32628    1.35307    1.28642
  0.153527   0.26925    0.339661  …   0.242358   0.283371   0.119573
  0.705142   0.651939   0.61815       0.668237   0.651582   0.68868
 -5.67186   -4.92102   -5.6336       -4.91027   -4.70509   -6.10014

In [36]:
@time model_gpu(input_gpu)

  0.002684 seconds (3.30 k allocations: 112.172 KiB)


8×100 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  1.60501    1.67968    1.59362   …   1.54868    1.55584    1.64938
 -1.22871   -1.25437   -1.31037      -1.25169   -1.29198   -1.28681
 -1.78119   -1.79311   -1.86135      -1.81473   -1.84307   -1.86589
 -0.554375  -0.498259  -0.433131     -0.511761  -0.484417  -0.557349
  1.29387    1.28261    1.42652       1.32628    1.35307    1.28642
  0.153525   0.26925    0.339661  …   0.242359   0.283371   0.119574
  0.705143   0.65194    0.618149      0.668237   0.651583   0.688681
 -5.67186   -4.92103   -5.6336       -4.91027   -4.7051    -6.10014